In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os
import numpy as np

In [ ]:
DATA_DIR = '../data/asl_alphabet_train/asl_alphabet_train'
TEST_DATA_DIR = '../data/asl_alphabet_test/asl_alphabet_test'

IMAGE_SIZE = (96, 96) 
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
SEED = 123

In [ ]:
print("Creating Training Dataset...")

train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

print("Creating Validation Dataset...")

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

print("Creating Test Dataset...")

class_names = train_ds.class_names
print(f"Found {len(class_names)} classes: {class_names}")

In [ ]:
print("Defining augmentation and normalization layers...")

data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomTranslation(0.1, 0.1),
        layers.RandomContrast(0.1),
    ],
    name="data_augmentation"
)

normalization_layer = layers.Rescaling(1./255) 

In [ ]:
print("Applying processing and optimizing datasets...")

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)

val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

print("Data processing pipeline complete and optimized.")

In [ ]:
print("Displaying augmented training batch...")

for images, labels in train_ds.take(1):
    plt.figure(figsize=(10, 10))
    plt.suptitle("Augmented Training Data (Batch 1)")
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy()) 
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis("off")
    plt.show()

print("Displaying un-augmented validation batch...")

for images, labels in val_ds.take(1):
    plt.figure(figsize=(10, 10))
    plt.suptitle("Un-Augmented Validation Data (Batch 1)")
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy())
        plt.title(class_names[np.argmax(labels[i])])
        plt.axis("off")
    plt.show()